# DATA PROJECT

In this project we will examine the inequality for disposable income for Denmark from 2010 to 2021 thorugh a Lorenz-Curve 

Imports and set magics:

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2
from scipy.stats import norm
###from matplotlib.widgets import Dropdown###
%matplotlib inline

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

Import your data, either through an API or manually, and load it. 

First we import the data to

In [23]:
LorenzData = "Indkomst Data.xlsx" #First we import the data
LData = pd.read_excel(LorenzData, skiprows=2) #Shorten the call for the data
LData.drop("Unnamed: 0", axis=1, inplace=True) #Removing first unnamed column
LData.rename(columns = {"Unnamed: 1":"Decil"}, inplace=True) #Renaming first column

LData = LData.dropna() #Removes missing values from bottom rows
LData.iloc[:, 1:] = LData.iloc[:, 1:].astype(int) #Removing decimal points from all numbers using iloc to locate numbers

LData


C:\Users\Asger Thimmer\AppData\Local\Temp\ipykernel_10604\648326278.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  LData.iloc[:, 1:] = LData.iloc[:, 1:].astype(int) #Removing decimal points from all numbers using iloc to locate numbers


,Decil,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1. decil,58517,61405,72143,75689,70459,77728,75853,78128,78721,83792,88188,95473
1,2. decil,133314,135247,138299,140042,142148,143630,144500,146503,151039,154985,160361,166971
2,3. decil,155258,157641,160792,163578,166795,169095,171292,174947,179714,184053,190111,196935
3,4. decil,176861,179613,182871,185928,190010,192715,195643,200087,205721,210708,218820,227383
4,5. decil,198711,202048,205633,209382,214412,217580,221437,226804,233247,238796,249210,259036
5,6. decil,220835,224771,228906,233521,239499,243288,248126,254422,261607,267917,280604,291453
6,7. decil,245061,249659,254534,260358,267373,271974,277775,285080,293222,300200,315119,327257
7,8. decil,274629,280171,285933,293589,301774,307454,314207,322842,331975,339935,357273,371556
8,9. decil,318054,325162,332513,342856,352557,360069,368072,378649,388934,398818,419353,437776
9,10. decil,500012,516944,533189,560971,573866,607665,615554,641188,648231,688377,719999,779827


## Explore each data set

In [24]:
Years = np.linspace(2010,2021,num=12) #Creating a list of numbers ranging from 2010-2021
Years_int = Years.astype(int) #Converting the list variables from float to integers
print(Years)

#Using for loop to calculate the cummulitative income for each year
for i in Years_int:
    Col_name = "Cum_" + str(i) #Looping over the names for the new columns
    LData[Col_name] = LData[str(i)].cumsum() #Calculating the cummulitative income

LData    
    

[2010. 2011. 2012. 2013. 2014. 2015. 2016. 2017. 2018. 2019. 2020. 2021.]


,Decil,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,Cum_2012,Cum_2013,Cum_2014,Cum_2015,Cum_2016,Cum_2017,Cum_2018,Cum_2019,Cum_2020,Cum_2021
0,1. decil,58517,61405,72143,75689,70459,77728,75853,78128,78721,...,72143,75689,70459,77728,75853,78128,78721,83792,88188,95473
1,2. decil,133314,135247,138299,140042,142148,143630,144500,146503,151039,...,210442,215731,212607,221358,220353,224631,229760,238777,248549,262444
2,3. decil,155258,157641,160792,163578,166795,169095,171292,174947,179714,...,371234,379309,379402,390453,391645,399578,409474,422830,438660,459379
3,4. decil,176861,179613,182871,185928,190010,192715,195643,200087,205721,...,554105,565237,569412,583168,587288,599665,615195,633538,657480,686762
4,5. decil,198711,202048,205633,209382,214412,217580,221437,226804,233247,...,759738,774619,783824,800748,808725,826469,848442,872334,906690,945798
5,6. decil,220835,224771,228906,233521,239499,243288,248126,254422,261607,...,988644,1008140,1023323,1044036,1056851,1080891,1110049,1140251,1187294,1237251
6,7. decil,245061,249659,254534,260358,267373,271974,277775,285080,293222,...,1243178,1268498,1290696,1316010,1334626,1365971,1403271,1440451,1502413,1564508
7,8. decil,274629,280171,285933,293589,301774,307454,314207,322842,331975,...,1529111,1562087,1592470,1623464,1648833,1688813,1735246,1780386,1859686,1936064
8,9. decil,318054,325162,332513,342856,352557,360069,368072,378649,388934,...,1861624,1904943,1945027,1983533,2016905,2067462,2124180,2179204,2279039,2373840
9,10. decil,500012,516944,533189,560971,573866,607665,615554,641188,648231,...,2394813,2465914,2518893,2591198,2632459,2708650,2772411,2867581,2999038,3153667


Generating 12 empty lists for the accumulative income for each year

In [25]:
# Setting the number of lists we want
Num_Lists = 12

# Namin our lists
List_Names = [f"Cum_Income_{i}" for i in Years_int]

# Creating a list where we can store our 12 lists as sublists
Empty_Lists = [[] for i in range(Num_Lists)]

for i, name in enumerate(List_Names):
    globals()[name] = Empty_Lists[i]


#Adding a 0 to each list if it is empty
for i in Empty_Lists :
    if i == [] :
        i.append(0)
    else :
        0 # Else do nothing


In [26]:
# If the list only holdes a 0 we calculate the percatage cummulitative income and putting them into the same list 
Cum_Income_List = []   
for l in Empty_Lists :    
    if i ==[0]:
        for i in range(13,25) :
            Denominator = LData.iloc[9,i] # Choosing the denominator in our dataset
            for j in range(10):
                Numerator = LData.iloc[j,i] # Choosing the numerator in our dataset
                Value = (Numerator/Denominator)*100 # Calculating the Value
                Num_Lists_Index = (i % Num_Lists) # Choosing which list to append the value to
                globals()[List_Names[Num_Lists_Index]].append(Value) # Appending the value to the above choosen list
    else :
        0    # Else do nothing
        

Making list with accumulative intervals for population

In [27]:
Total = 0 # Setting a start variable with value 0

Cum_Num = [] # Creating empty list

Cum_Num.append(Total) # Appending 0 to the list

# Creating a loop that adds the numbers from 0 to 100 in intervals in 10
for i in range(10): 
    Total += 10
    Cum_Num.append(Total)

print(Cum_Num) # Printing the list


[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


In [28]:
# Creating zips with the accumulative values

Zipped_Lists = [] # Making empty list

# Creating for loop that zips each of our 12 lists with the list "Cum_Num" and appending them to a new list
for l in Empty_Lists:
    Zipped_List = list(zip(Cum_Num, l))
    Zipped_Lists.append(Zipped_List)
    

# Plotting the data

In [29]:
# Defining our diagram in a function so that we can add a slider widget
def L_Diagrams(Empty_Lists, Year):
    # Creating the figure
    fig = plt.figure()

    # Making plot
    ax = fig.add_subplot(1,1,1)

    # We want our slider to go from 2010 to 2021, so we set our List_Choosing to "Year - 2010", this way it can still choose the correct list by indexing 0 to 11
    List_Choosing = Empty_Lists[Year - 2010]
    ax.plot(Cum_Num,List_Choosing, 'o-', label="Lorenz Curve")
    ax.plot(Cum_Num,Cum_Num, label="45 degree line")

    # setting labels and title
    ax.set_title('Lorenz Diagram')
    ax.set_xlabel('Share of population')
    ax.set_ylabel('Accumulative disposable income');

    # Adding a grid
    ax.grid(True)

    # Setting limits on each axis
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

    # Setting legend
    ax.legend(['Lorenz Curve', '45 degree line'])

    # Creating a for loop to show the values on our Lorenz Curve
    for x, y in zip(Cum_Num, List_Choosing):
        # Setting the label and the decimals
        Values = f"({x:.2f}, {y:.2f})"
        ax.annotate(Values, # Choosing the values to display
                    xy=(x, y), 
                    textcoords="offset points", # Choosing where to show the pount
                    xytext=(5,5), # Choosing the distance from text to points
                    ha='left') # Setting alignment


# Making our widget slider
interactive_plot = widgets.interact(L_Diagrams, # Choosing the diagram
                 Empty_Lists=widgets.fixed(Empty_Lists), # Setting what variables the slider needs to choose
                 
                 Year=widgets.IntSlider( # Setting the parameters and names for the widget
                     description="Year", 
                     # Min, max and value is set to the corresponding years of our data
                     min=2010, 
                     max=2021, 
                     step=1, 
                     value=2010, 
                     continuous_update=True, 
                     readout=True 
                     )               
)




plt.show() 

interactive(children=(IntSlider(value=2010, description='Year', max=2021, min=2010), Output()), _dom_classes=(…

# Comparing income groups 

In this section there is created a graph with a widget slider to compare different income groups. 

In [30]:
income_df = LData.iloc[:, 1:13]  # Creating a list for all of the income groups only
decil_list = LData.Decil.tolist()  # Making a list of the decils

In [31]:

def L_Diagrams(df, decils):
    # Creating the figure 
    fig = plt.figure()
    # Making the plot 
    ax = fig.add_subplot(1,1,1)
    
    for d in decils:
        i = decil_list.index(d)
        income = income_df.iloc[i]
        ax.plot(income, label=decil_list[i])
    
    ax.legend()
    plt.show()

# Making our widget slider 
interactive_plot = widgets.interact(
    L_Diagrams, #Choosing the diagram
    df=widgets.fixed(income_df),  # Setting what variables the slider needs to choose
    decils=widgets.SelectMultiple(
        options=decil_list, 
        value=["1. decil"]
    )
)



interactive(children=(SelectMultiple(description='decils', index=(0,), options=('1. decil', '2. decil', '3. de…

# Conclusion

A Lorenz Diagram contains the Lorenz Curve (the blue curve) plotted in a diagram against a 45 degree line (the orange curve). It is made to show how much inequality there is for a group of people given a type of wealth measurement. We have chosen to measure the inequality in disposable income for Denmark in the periode from 2010 to 2021.

The diagram is made my calculating the accumulative population and plotting that on the x-axis, while plotting the accumulative disposable income on the y-axis. Both being shown in percentages. By doing this it is clear shown how much of the wealth for example belongs to a given percantage of the population. 

If there is perfect distribution of wealth the Lorenz Curve will lie on the 45 degree line. If there is perfect inequality (1 person has all the wealth) it will follow the outer axes. 

From our Lorenz Diagram above showing the development in the inequality from the periode 2010 to 2021, we see that there has not been that much of an evolution. We see that in the periode 2010 to 2013 the top decile of the population got less disposable income and moved closer to the 45 degree line. 
In the following periode the inequality grew a small amount and they ended up almost at the same place as in 2010. This fall in inequality could also be seen by the 60% of the population moved closer to the 45 degree line. 

In the last section we have created a plot where you can choose different income groups and compare them (by holding CTRL while clicking on the drop-down menu). If we compare all of the groups the general trend from 2010 to 2021 is that "the rich gets richer". The higher decil income group you are in the steeper the curve. The plot therefore also shows a trend of higher inequaity through the years. Consindering the principle of compounded interest this overeall makes good sense. 
 